In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pydicom as pyd
from skimage.transform import resize
from skimage import measure
from tensorflow import keras
import random

client = MongoClient("localhost", 27017)
db = client["rsna-xray"]
patients = db['patient']

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
class generator(keras.utils.Sequence):
    
    def __init__(self, documents, batch_size = 32, img_size=256, 
                shuffle = True, augment = False, predict = False):
        #intializing object

        self.documents = documents
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.augment = augment
        self.predict = predict
        self.on_epoch_end()
    
    def __load__(self, document):
        #loading the image
        img = pyd.read_file(document["path"]).pixel_array
        #creating a mask of zeros
        mask = np.zeros(img.shape)
        #getting if the patient contains puenomia
        if document["target"] == 1:
            #For each of the bounding boxes
            for box in document["targets"]:
                #Unpacking the box
                x,y,w,h = box
                #putting 1's in the bounding box
                mask[y:y+h, x:x+w] = 1
        #making the image the specified shape
        img = resize(image=img, output_shape = (self.img_size, self.img_size), mode='reflect')
        #Adjusting the mask size
        mask = resize(image=mask, output_shape = (self.img_size, self.img_size), mode = 'reflect')
        #Augmentation can create more training samples
        if self.augment and np.random.uniform() > 0.5:
            img = np.fliplr(img)
            msk = np.fliplr(mask)
        img = np.expand_dims(img, -1)
        mask = np.expand_dims(mask, -1)
    
    def __loadpredict__(self, document):
        # load image
        img = pyd.read_file(document["path"]).pixel_array
        # resize image
        img = resize(img, output_shape = (self.img_size, self.img_size), mode='reflect')
        # add trailing channel dimension
        img = np.expand_dims(img, -1)
        return img
    
    def __getitem__(self, index):
        # select batch
        documents = self.documents[index*self.batch_size:(index+1)*self.batch_size]
        # predict mode: return images and filenames
        if self.predict:
            # load files
            imgs = [self.__loadpredict__(document) for document in documents]
            # create numpy batch
            imgs = np.array(imgs)
            return imgs, documents["patientID"]
        # train mode: return images and masks
        else:
            # load files
            items = [self.__load__(document) for document in documents]
            # unzip images and masks
            imgs, masks = zip(*items)
            # create numpy batch
            imgs = np.array(imgs)
            masks = np.array(masks)
            return imgs, masks
    
    def on_epoch_end(self):
        if self.shuffle:
            random.shuffle(self.documents)
        
    def __len__(self):
        if self.predict:
            # return everything
            return int(np.ceil(len(self.documents) / self.batch_size))
        else:
            # return full batches only
            return int(len(self.documents) / self.batch_size)
        
                
    

In [8]:
def create_downsample(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 1, padding='same', use_bias=False)(x)
    x = keras.layers.MaxPool2D(2)(x)
    return x

def create_resblock(channels, inputs):
    x = keras.layers.BatchNormalization(momentum=0.9)(inputs)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(x)
    return keras.layers.add([x, inputs])

def create_network(input_size, channels, n_blocks=2, depth=4):
    # input
    inputs = keras.Input(shape=(input_size, input_size, 1))
    x = keras.layers.Conv2D(channels, 3, padding='same', use_bias=False)(inputs)
    # residual blocks
    for d in range(depth):
        channels = channels * 2
        x = create_downsample(channels, x)
        for b in range(n_blocks):
            x = create_resblock(channels, x)
    # output
    x = keras.layers.BatchNormalization(momentum=0.9)(x)
    x = keras.layers.LeakyReLU(0)(x)
    x = keras.layers.Conv2D(1, 1, activation='sigmoid')(x)
    outputs = keras.layers.UpSampling2D(2**depth)(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [9]:
# define iou or jaccard loss function
def iou_loss(y_true, y_pred):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    score = (intersection + 1.) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection + 1.)
    return 1 - score

# combine bce loss and iou loss
def iou_bce_loss(y_true, y_pred):
    return 0.5 * keras.losses.binary_crossentropy(y_true, y_pred) + 0.5 * iou_loss(y_true, y_pred)

# mean iou as a metric
def mean_iou(y_true, y_pred):
    y_pred = tf.round(y_pred)
    intersect = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3])
    union = tf.reduce_sum(y_true, axis=[1, 2, 3]) + tf.reduce_sum(y_pred, axis=[1, 2, 3])
    smooth = tf.ones(tf.shape(intersect))
    return tf.reduce_mean((intersect + smooth) / (union - intersect + smooth))

# create network and compiler
model = create_network(input_size=256, channels=32, n_blocks=2, depth=4)
model.compile(optimizer='adam',
              loss=iou_bce_loss,
              metrics=['accuracy', mean_iou])

# cosine learning rate annealing
def cosine_annealing(x):
    lr = 0.001
    epochs = 25
    return lr*(np.cos(np.pi*x/epochs)+1.)/2

learning_rate = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

In [10]:
# create train and validation generators
pat = [patient for patient in patients.find()]
train_gen = generator(pat, batch_size = 32, img_size=256, shuffle = True, augment = True, predict = False)
valid_gen = generator(pat, batch_size = 32, img_size=256, shuffle = True, augment = True, predict = False)
history = model.fit_generator(train_gen, validation_data=valid_gen, callbacks=[learning_rate], epochs=25)

Epoch 1/25


StopIteration: slice indices must be integers or None or have an __index__ method